# Unterscheide Gedichte von Spam - naiver Ansatz

In [ ]:
import string
import numpy as np
import pandas as pd
import sklearn.tree
import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix
import collections

## Lade Daten

In [ ]:
df_spam = pd.read_csv("spam_betreff_und_text.csv", index_col=0)
df_spam

In [ ]:
df_poems = pd.read_csv("poems.csv", index_col=0)
df_poems

Führe die zwei Datensätze zusammen.

In [ ]:
df_poems_merger = df_poems.copy()
df_poems_merger = df_poems_merger.assign(category="poem")
df_poems_merger.columns = ["creator", "title", "text", "category"]

In [ ]:
df_spam_merger = df_spam.copy()
df_spam_merger = df_spam_merger.assign(category="spam")
df_spam_merger.columns = ["creator", "title", "text", "category"]

In [ ]:
df = pd.concat([df_poems_merger, df_spam_merger])
df

In [ ]:
df.info()

Remove rows with missing values

In [ ]:
df = df.dropna()
df

## Feature Engineering

Es wird für jeden Eintrag ein Vektor $x$ erzeugt.
Die meisten ML-Verfahren können nur Zahlenwerte in Form von Vektoren und Matrizen verarbeiten, weswegen Texte speziell aufbereitet werden müssen.

In [ ]:
features = []

for i, row in df.iterrows():
    features.append({
        "category": row["category"],
        "Textlänge": len(row["text"]),
        "Anzahl 'Geld'": row["text"].lower().count("money") + row["text"].lower().count("geld"),
        "Anzahl '!'": row["text"].lower().count("!"),
        "Großbuchstaben": (len([x for x in row["text"] if x in string.ascii_uppercase]) /
                           len([x for x in row["text"] if x in string.ascii_letters]))
    })

df_text_features = pd.DataFrame(features)
df_text_features

## Teile Daten auf

In [ ]:
df_text_features_only = df_text_features.drop("category", axis=1)

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(
    df_text_features_only.values, df_text_features["category"].values,
    test_size=0.33, random_state=42
)

X_train = np.stack(X_train, axis=0)
X_test = np.stack(X_test, axis=0)

## Trainiere Entscheidungsbaum

In [ ]:
clf = sklearn.tree.DecisionTreeClassifier(max_depth=10)
clf = clf.fit(X_train, y_train)

Berechne Accuracy-Wert für den Trainings-Datensatz.
Diese Daten kennt der Lernalgorithmus schon.

In [ ]:
clf.score(X_train, y_train)

## Untersuche Ergebnis

Berechne Accuracy-Wert für den Test-Datensatz.
Dies zeigt, wie gut sich die Daten verallgemeinern lassen.

In [ ]:
clf.score(X_test, y_test)

In [ ]:
plot_confusion_matrix(clf, X_test, y_test, cmap="BuPu")

In [ ]:
y_train_counter = collections.Counter(y_train)
display(y_train_counter)
sorted_class_names_with_counts = list(reversed(sorted(y_train_counter.items(), key=lambda x: x[1])))
display(sorted_class_names_with_counts)
sorted_class_names = [el[0] for el in sorted_class_names_with_counts]
sorted_class_names

Zur Erinnerung:
In der obersten Zeile steht mit `<=` der Vergleich, nach dem nach links (zutreffend) und rechts (nicht zutreffend) aufgeteilt wird.

In [ ]:
plt.figure(figsize=(27, 10))
sklearn.tree.plot_tree(
    clf,
    feature_names=df_text_features_only.columns,
    class_names=sorted_class_names  # Dokumentation: "Names of each of the target classes in ascending numerical order"
)
plt.show()

<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Creative Commons Lizenzvertrag" style="border-width:0; display:inline" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a> &nbsp;&nbsp;&nbsp;&nbsp;Dieses Werk von Marvin Kastner ist lizenziert unter einer <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Creative Commons Namensnennung 4.0 International Lizenz</a>.